## 44 rows dont have cricinfo 1, 2, 3.
## no rows which have 1 null not not 2 null
## 92 rows where 2 is not null but 3 is.
## no rows where 3 is not null but 2 is
## 16119 null-2
## 16211 null-3

In [18]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ProcessPoolExecutor
import pandas as pd
from tqdm import tqdm

all_players = pd.read_csv("../data/raw/cricsheet/people.csv")

all_players = all_players[~all_players["key_cricinfo"].isna()]
all_players.key_cricinfo = all_players.key_cricinfo.astype("Int64")
all_players["key_cricinfo_2"] = all_players["key_cricinfo_2"].astype("Int64")


url = 'https://www.espncricinfo.com/ci/content/player/{}.html'

In [19]:
from requests_html import AsyncHTMLSession
import asyncio
import nest_asyncio
nest_asyncio.apply()

asession = AsyncHTMLSession()

async def fetch_html(url, headers):
    response = await asession.get(url, headers=headers)
    await response.html.arender(timeout=20)  # async render
    return response

# helper to run async code inside Jupyter
def run_async_fetch(url, headers):
    return asyncio.get_event_loop().run_until_complete(fetch_html(url, headers))


In [22]:
# get full name, Batting style, Bowling style, Playing role
def get_data(paragraphs, data, id):
    try:
        p_index = [i for i, p in enumerate(paragraphs) if data in p.text][0]
        res = paragraphs[p_index+1].text
    except IndexError:
        res = None
    return res

# make a new dataframe with the above data for each player along with Key_cricinfo column
def process_row_data(row):
    id = row['key_cricinfo']

    # header = {
    #     "accept": "*/*",
    #     "accept-encoding": "gzip, deflate, br",
    #     "accept-language": "en-US,en;q=0.9",
    #     "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    # }

    header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Referer": "https://www.google.com/",
        "DNT": "1",  # Do Not Track header
        "Upgrade-Insecure-Requests": "1",
    }
    print(url.format(id))
    response = run_async_fetch(url.format(id), headers=header)
    if response.status_code != 200:
        if response.status_code == 404:
            id = row["key_cricinfo_2"]
            response = run_async_fetch(url.format(id), headers=header)
            if response.status_code != 200:
                print("Failed to fetch the webpage. Status code:", response.status_code, "for player ID:", id, "1", row["identifier"])
                return [id, None, None]
        
        else:
            print("Failed to fetch the webpage. Status code:", response.status_code, "for player ID:", id, "2", row["identifier"])
            return [id, None, None]

    soup = BeautifulSoup(response.text, 'html.parser')

    paragraphs = soup.find_all('p')
    
    # full_name = get_data(paragraphs, 'Full Name', id)
    # batting_style = get_data(paragraphs, 'Batting Style', id)
    # bowling_style = get_data(paragraphs, 'Bowling Style' , id)
    playing_role = get_data(paragraphs, 'Playing Role' , id)
    names = soup.find_all("h1", attrs={"class": "ds-text-title-l ds-font-bold"})
    if len(names) != 1:
        print(id, "name is not 1")
    cricinfo_name = names[0]
    return [id, playing_role]

In [23]:
# with ProcessPoolExecutor(max_workers=1) as executor:
#     result = list(tqdm(executor.map(process_row_data, [row for _, row in all_players.iloc[:, :].iterrows()]), total=len(all_players), desc="Processing data"))

print(process_row_data(all_players.iloc[0, :]))
# print(result[:10])

# pd.DataFrame(result).to_csv("player_roles.csv", index=False)

https://www.espncricinfo.com/ci/content/player/772407.html


/home/ai21btech11012/miniconda3/envs/dream11/lib/python3.10/site-packages/pyppeteer/util.py:29: RuntimeWarning: coroutine 'fetch_html' was never awaited
  gc.collect()


Failed to fetch the webpage. Status code: 403 for player ID: 772407 2 b4a23876
[np.int64(772407), None, None]


In [33]:
import requests
from bs4 import BeautifulSoup

def fetch_player_info(player_id):
    url = f"https://www.espncricinfo.com/ci/content/player/{player_id}.html"
    
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "max-age=0",
        "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": '"Windows"',
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    }

    response = requests.get(url, headers=headers)
    print(response)

    if response.status_code != 200:
        print(f"❌ Failed for {player_id} with status code {response.status_code}")
        print(response.text)  # Print response text for debugging
        return None, None

    soup = BeautifulSoup(response.text, 'html.parser')

    # Player name
    name_tag = soup.find("h1", class_="ds-text-title-l ds-font-bold")
    player_name = name_tag.text.strip() if name_tag else None

    # Player info paragraphs
    paragraphs = soup.find_all("p")
    def get_data(label):
        try:
            idx = [i for i, p in enumerate(paragraphs) if label in p.text][0]
            return paragraphs[idx+1].text.strip()
        except:
            return None

    playing_role = get_data("Playing Role")

    return player_name, playing_role

# Test the function
name, role = fetch_player_info(772407)
print("Name:", name)
print("Role:", role)

<Response [403]>
❌ Failed for 772407 with status code 403
<HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;www&#46;espncricinfo&#46;com&#47;ci&#47;content&#47;player&#47;772407&#46;html" on this server.<P>
Reference&#32;&#35;18&#46;5f6e3f17&#46;1744463232&#46;fa4b3e7
<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;5f6e3f17&#46;1744463232&#46;fa4b3e7</P>
</BODY>
</HTML>

Name: None
Role: None


In [1]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from bs4 import BeautifulSoup

def fetch_player_info(player_id):
    url = f"https://www.espncricinfo.com/ci/content/player/{player_id}.html"
    
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "max-age=0",
        "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": '"Windows"',
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    }
    driver = webdriver.Chrome()  # Requires ChromeDriver
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    # response = requests.get(url, headers=headers)
    # print(response)

    # if response.status_code != 200:
    #     print(f"❌ Failed for {player_id} with status code {response.status_code}")
    #     print(response.text)  # Print response text for debugging
    #     return None, None

    # soup = BeautifulSoup(response.text, 'html.parser')

    # Player name
    name_tag = soup.find("h1", class_="ds-text-title-l ds-font-bold")
    player_name = name_tag.text.strip() if name_tag else None

    # Player info paragraphs
    paragraphs = soup.find_all("p")
    def get_data(label):
        try:
            idx = [i for i, p in enumerate(paragraphs) if label in p.text][0]
            return paragraphs[idx+1].text.strip()
        except:
            return None

    playing_role = get_data("Playing Role")

    return player_name, playing_role

# Test the function
name, role = fetch_player_info(772407)
print("Name:", name)
print("Role:", role)

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir
Stacktrace:
#0 0x56423cf93d0a <unknown>
#1 0x56423ca445f0 <unknown>
#2 0x56423ca7e0db <unknown>
#3 0x56423ca79e2f <unknown>
#4 0x56423cac9f9e <unknown>
#5 0x56423cac94c6 <unknown>
#6 0x56423cabb433 <unknown>
#7 0x56423ca87ea3 <unknown>
#8 0x56423ca88b01 <unknown>
#9 0x56423cf58b5b <unknown>
#10 0x56423cf5ca41 <unknown>
#11 0x56423cf3fc52 <unknown>
#12 0x56423cf5d5b4 <unknown>
#13 0x56423cf23f0f <unknown>
#14 0x56423cf81db8 <unknown>
#15 0x56423cf81f96 <unknown>
#16 0x56423cf92b56 <unknown>
#17 0x7f2b741a6609 start_thread


In [32]:
name, role = fetch_player_info(772407)
print("Name:", name)
print("Role:", role)


<Response [403]>
❌ Failed for 772407 with status code 403
Name: None
Role: None
